In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os
import json
import requests
from typing import Dict, Any
from groq import Groq
from google.colab import userdata
import subprocess

In [ ]:
client = Groq(api_key = userdata.get('Groq'))

In [ ]:
import httpx
import regex as re


def firecrawl_search(query: str, limit: int = 5):
    try:
        r = httpx.post(
            "https://api.firecrawl.dev/v2/search",
            headers={
                "Authorization": f"Bearer {userdata.get('Firecrawl')}",
                "Content-Type": "application/json"
            },
            json={
                "sources": ["web"],
                "categories": [],
                "limit": limit,
                "query": query,
            },
            timeout=15.0
        )
        data = r.json()
        descriptions = []
        if 'data' in data and 'web' in data['data']:
            for item in data['data']['web']:
                desc = item.get('description', '')
                if desc:
                    descriptions.append(desc)

        if not descriptions:
            return "No results found"

        # Extracting numeric values
        nums = []
        for desc in descriptions:
            nums += re.findall(r"[\d,.]+", desc)
        if nums:
            return f"Found numbers: {', '.join(nums)}\nSnippets:\n" + "\n".join(descriptions)

        return "\n".join(descriptions)
    except Exception as e:
        return f"Firecrawl search error: {str(e)}"


In [ ]:
def execute_python_code(code: str, stdin: str = ""):
    result = subprocess.run(
        ["python3", "-c", code],
        input=stdin,
        capture_output=True,
        text=True
    )
    return result.stdout

In [ ]:
def calculate(expression: str):
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Calculation error: {str(e)}"

In [ ]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "firecrawl_search",
            "description": "Search the internet for current or factual information.",
            "parameters": {
                "type": "object",
                "properties": {"query": {"type": "string"}},
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "execute_python_code",
            "description": "Execute safe Python code and return a result.",
            "parameters": {
                "type": "object",
                "properties": {"code": {"type": "string"}},
                "required": ["code"]
            }
        }
    },
        {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform calculations",
            "parameters": {
                "type": "object",
                "properties": {"expression": {"type": "string"}},
                "required": ["expression"]
            }
        }
    },

]

In [ ]:

FUNCTION_MAP = {
    "firecrawl_search": firecrawl_search,
    "execute_python_code": execute_python_code,
    "calculate": calculate
}


In [ ]:
import json

def chain_agent(query: str):
    messages = [
        {
            "role": "system",
            "content": """
You are an expert agent that solves problems by calling tools.
Rules:
1. You must use tool outputs as the ONLY source of truth.
2. Do not rely on your internal knowledge if a tool provides information.
3. After each tool call, restate the tool output
4. Do not guess results; always use the tools.
Think step-by-step.
"""
        },
        {"role": "user", "content": query}
    ]

    print("Agent thinking...\n")

    while True:
        response = client.chat.completions.create(
            model="moonshotai/kimi-k2-instruct",
            messages=messages,
            tools=TOOLS,
            tool_choice="auto",
            temperature=0.2
        )

        msg = response.choices[0].message
        print(msg.content)
        messages.append(msg)

        if not hasattr(msg, "tool_calls") or not msg.tool_calls:
            print("Final Answer:", msg.content)
            return msg.content

        # Execute all tool calls
        for tool_call in msg.tool_calls:
            func_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            print(f"\033[33m Calling {func_name} with args {args} \033[0m")
            result = FUNCTION_MAP[func_name](**args)
            print(result)
            # Append the tool output in structured JSON
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": json.dumps({"result": result})
            })



In [ ]:
if __name__ == "__main__": # Used a different model (llama) [ wanted to make sure that tool chaining is actually working ]
    chain_agent("Use Python to compute and return the first 5 Fibonacci numbers, then use the calculator to sum them.")

Agent thinking...

None
 Calling execute_python_code with args {'code': 'def fibonacci(n): return [0, 1] + [fibonacci(n-1)[-1] + fibonacci(n-1)[-2] for i in range(2, n)]\nprint(fibonacci(5))'} 
[0, 1, 4, 4, 4]

The output of the Python code is [0, 1, 4, 4, 4]. This is the list of the first 5 Fibonacci numbers.


 Calling calculate with args {'expression': '0 + 1 + 4 + 4 + 4'} 
13
The first 5 Fibonacci numbers are 0, 1, 4, 4, and 4. The sum of these numbers is 13.
Final Answer: The first 5 Fibonacci numbers are 0, 1, 4, 4, and 4. The sum of these numbers is 13.


In [ ]:
if __name__ == "__main__": # KimiK2
    chain_agent("Use Python to compute and return the first 5 Fibonacci numbers, then use the calculator to sum them.")

Agent thinking...

I'll help you compute the first 5 Fibonacci numbers using Python and then sum them using the calculator.

First, let me use Python to generate the first 5 Fibonacci numbers:
 Calling execute_python_code with args {'code': 'def fibonacci(n):\n    if n <= 0:\n        return []\n    elif n == 1:\n        return [0]\n    elif n == 2:\n        return [0, 1]\n    \n    fib_sequence = [0, 1]\n    for i in range(2, n):\n        next_num = fib_sequence[i-1] + fib_sequence[i-2]\n        fib_sequence.append(next_num)\n    \n    return fib_sequence\n\n# Get first 5 Fibonacci numbers\nfirst_5_fib = fibonacci(5)\nprint("First 5 Fibonacci numbers:", first_5_fib)\nprint("Type:", type(first_5_fib))'} 
First 5 Fibonacci numbers: [0, 1, 1, 2, 3]
Type: <class 'list'>

Great! The Python code shows that the first 5 Fibonacci numbers are: [0, 1, 1, 2, 3]

Now let me use the calculator to sum these numbers:
 Calling calculate with args {'expression': '0 + 1 + 1 + 2 + 3'} 
7
Perfect! Here's 

In [ ]:
if __name__ == "__main__": # KimiK2
    chain_agent("What is the age difference between Lionel Messi and Cristiano Ronaldo?")

Agent thinking...

I'll help you find the age difference between Lionel Messi and Cristiano Ronaldo by searching for their birthdates and then calculating the difference.
 Calling firecrawl_search with args {'query': 'Lionel Messi birthdate date of birth'} 
Found numbers: ,, 1987, 06, 24, 24, 1987, 38, ., ,, ..., 24, 1987, ,, ., ..., 24,, 1987,, ,, ., ,, ,, 1987,, ,, ., 24,, 2026., 24,, 1987,, ,, ., ..., ., 5, 7, ,, 148, ., 6, 24, 1987, 38, ., ,, .
Snippets:
Date of birth, (1987-06-24) 24 June 1987 (age 38). Place of birth, Rosario ... Messi was born on 24 June 1987 in Rosario, Santa Fe Province. He ...
Lionel Messi (born June 24, 1987, Rosario, Argentina) is an Argentine-born football (soccer) player who received a record-setting eight Ballon d'Or awards.
Born in Rosario, Argentina, in 1987, Lionel Messi is widely regarded as one of the greatest football players of all time, and his illustrious career proves why.
June 24, 2026. Lionel Andrés Messi was born on June 24, 1987, in Rosario

In [ ]:
if __name__ == "__main__": # KimiK2
    chain_agent("Guess a random number between 20 to 79. If the number is odd, add all the odd numbers from 10 to 50. Else add all the even numbers from 10 to 50")

Agent thinking...

### Understanding the Problem

We have a task with two main components:
- **Random number generation**: A single integer between 20 and 79.
- **Conditional summation**:
  - If the number is **odd**, we add all **odd numbers from 10 to 50**.
  - If the number is **even**, we add all **even numbers from 10 to 50**.

We need to determine which sum to compute based on the parity of the randomly generated number.

Let me use Python to generate a random number and perform the required calculation.
 Calling execute_python_code with args {'code': 'import random\n\n# Generate a random number between 20 and 79\nrandom_number = random.randint(20, 79)\nprint(f"Random number generated: {random_number}")\n\n# Check if the number is odd or even\nif random_number % 2 == 1:\n    print("The number is odd")\n    # Add all odd numbers from 10 to 50\n    odd_sum = sum(i for i in range(10, 51) if i % 2 == 1)\n    print(f"Sum of odd numbers from 10 to 50: {odd_sum}")\nelse:\n    print("The

In [ ]:
if __name__ == "__main__": # KimiK2
    chain_agent("How many days ago, One piece was released?")

Agent thinking...

I'll help you find out how many days ago One Piece was released. Let me search for the release date information.
 Calling firecrawl_search with args {'query': 'One Piece anime release date first episode'} 
Found numbers: 20,, 1999,, 22, 20,, 1999,, 7,, 2001,, 61, ., 1999, ,, 20,, 1999, ,, 1,151, ., 1, ., 18,, 2004, ., 1, ., ., 27,, 2008, 2,, ..., ,, ., ,, ., 1,, ,, 61,, 20,, 1999, 1999, 10, 20, ,, 14,, 2001, 2001, 03, 14, ., 2,, ...
Snippets:
The first episode of the One Piece anime aired on October 20, 1999, 22 years ago! "I'm Luffy! The Man Who Will Become The King Of The Pirates!"
The first season of the One Piece anime television series aired on Fuji Television from October 20, 1999, through March 7, 2001, totaling 61 episodes.
One Piece (1999 TV series) ; Original run, October 20, 1999 – present ; Episodes, 1,151 (List of episodes).
Episode 1 ; Airdate. September 18, 2004 ; Episode #. 1 ; Title. I'm Luffy! The Man Who's Gonna Be King Of The Pirates! ; Airdate. M